# 🏭 Supplier Evaluation & Clustering
This notebook trains a clustering model to evaluate suppliers based on their procurement KPIs.  
We will:
1. Load and clean the dataset.
2. Aggregate KPIs per supplier.
3. Scale data.
4. Find the optimal number of clusters (`k`).
5. Train the final clustering model.
6. Compute a composite supplier score.
7. Save the model and results for future use.

---


In [11]:
# --- Imports ---
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import joblib
from pathlib import Path


## ⚙️ Step 1: Configuration
Here we define key parameters, including:
- Input Excel file and sheet name.
- Output directory to save model and results.
- Random state for reproducibility.


In [12]:
# --- CONFIG ---
INPUT_FILE = "Procurement KPI Analysis Dataset 1.xlsx"
SHEET_NAME = "Procurement KPI Analysis Datase"
OUTPUT_DIR = Path("supplier_evaluation_output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_STATE = 42


## 📂 Step 2: Load Data & Define Search Range
1) Load the procurement dataset and determine the number of unique suppliers.
2) Then, set K_SEARCH_RANGE from 2 up to the total number of suppliers.


In [13]:
# --- Load Data ---
df = pd.read_excel("Procurement KPI Analysis Dataset 1.xlsx", sheet_name="Procurement KPI Analysis Datase")
df.head()


,PO_ID,Supplier,Order_Date,Delivery_Date,Item_Category,Order_Status,Quantity,Unit_Price,Negotiated_Price,Compliance
0,PO-00001,Alpha_Inc,2023-10-17,2023-10-25,Protein,Cancelled,1176,20.13,17.81,Yes
1,PO-00002,Delta_Logistics,2022-04-25,2022-05-05,Vitamin,Delivered,1509,39.32,37.34,Yes
2,PO-00003,Gamma_Co,2022-01-26,2022-02-15,Omega,Delivered,910,95.51,92.26,Yes
3,PO-00004,Beta_Supplies,2022-10-09,2022-10-28,Vitamin,Delivered,1344,99.85,95.52,Yes
4,PO-00005,Delta_Logistics,2022-09-08,2022-09-20,Performance,Delivered,1180,64.07,60.53,No


In [14]:
# --- Determine dynamic search range for clusters ---
n_suppliers = df["Supplier"].nunique()
n_suppliers

5

In [15]:
# k must be between 2 and (n_suppliers - 1)
# max(3, n_suppliers) ensures there are at least 2 valid cluster options.
K_SEARCH_RANGE = range(2, max(3, n_suppliers))  #(2,3,...,n_suppliers-1)

## 📊 Step 3: Aggregate KPIs per Supplier
We group by `Supplier` and calculate:
- **Compliance Rate:** % of orders marked as "Yes" in the Compliance column.  
- **Delivery Rate:** Average of `OnTimeDelivery` values.  
- **Average Price Efficiency:** Mean `UnitPrice`.  
- **Quantity Share:** Total quantity of each supplier divided by total quantity overall.  
- **Total Orders & Total Quantity** for additional reference.


In [16]:
# --- Aggregate per supplier ---
agg = df.groupby("Supplier").agg(
    compliance_rate=("Compliance", lambda x: (x == "Yes").mean()),
    delivery_rate=("Order_Status", lambda x: (x == "Delivered").mean()),
    avg_price_efficiency=("Unit_Price", "mean"),   # Unit price used as proxy
    quantity_share=("Quantity", "sum"),
    total_orders=("PO_ID", "count"),
    total_quantity=("Quantity", "sum")
).reset_index()

In [17]:
# Normalize quantity share
agg["quantity_share"] = agg["quantity_share"] / agg["quantity_share"].sum()
agg.head()

,Supplier,compliance_rate,delivery_rate,avg_price_efficiency,quantity_share,total_orders,total_quantity
0,Alpha_Inc,0.936170,0.758865,58.386525,0.176843,141,150414
1,Beta_Supplies,0.756410,0.705128,58.279423,0.212234,156,180516
2,Delta_Logistics,0.608187,0.701754,56.236023,0.213570,171,181652
3,Epsilon_Group,0.981928,0.722892,59.392711,0.211690,166,180053
4,Gamma_Co,0.860140,0.720280,59.348881,0.185663,143,157916


## 🔄 Step 4: Feature Scaling
Standardize the features to have **mean = 0** and **variance = 1** to ensure all KPIs are on the same scale.


In [18]:
# --- Normalize numeric features ---
features = ["compliance_rate", "delivery_rate", "avg_price_efficiency", "quantity_share"]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(agg[features])

## 🔍 Step 5: Finding the Optimal `k`
We test different values of `k` (2 to 6 or fewer if dataset is small)  
using the **Silhouette Score** to choose the best number of clusters.


In [19]:
# --- Find best number of clusters ---
best_k = None
best_score = -999
scores = {}
for k in K_SEARCH_RANGE:
    km_tmp = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels_tmp = km_tmp.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels_tmp)
    db = davies_bouldin_score(X_scaled, labels_tmp)

    scores[k] = {"silhouette": sil, "davies_bouldin": db}
    if sil > best_score:
        best_score = sil
        best_k = k

print("Cluster selection results:")
for k, val in scores.items():
    print(f" k={k}: silhouette={val['silhouette']:.4f}, davies_bouldin={val['davies_bouldin']:.4f}")
print(f"✅ Selected k = {best_k}")

Cluster selection results:
 k=2: silhouette=0.3185, davies_bouldin=0.7771
 k=3: silhouette=0.1764, davies_bouldin=0.6280
 k=4: silhouette=0.0958, davies_bouldin=0.3166
✅ Selected k = 2


## 📊 Step 6: Train Final KMeans Model & Assign Clusters
We train the final KMeans clustering model using the best value of `k`.  
The resulting cluster labels are then added to the aggregated dataset.  


In [ ]:
# --- Fit final KMeans ---
final_kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=20)

# This line trains the final KMeans model on the reduced dataset (X_reduced) 
# and assigns a cluster number (label) to each supplier/data point.
final_labels = final_kmeans.fit_predict(X_reduced)

# This line creates a new column "cluster" in your agg DataFrame 
# and stores the cluster label (from final_labels) for each supplier.
agg["cluster"] = final_labels

# Shows the first 5 rows of agg with the new "cluster" column
agg.head()


NameError: name 'X_reduced' is not defined

## 🏆 Step 7: Compute Composite Supplier Score & Ranking

We calculate a **composite score** to evaluate suppliers by combining their KPIs with defined weights:

* **Compliance Rate:** 40%
* **Delivery Rate:** 30%
* **Price Efficiency:** 20% *(inverted so lower prices are better)*
* **Quantity Share:** 10% *(normalized using percentile ranking)*

The final `supplier_score` reflects overall supplier performance.
We then generate a `rank` column to order suppliers from **best to worst** and display the **Top 10 suppliers** for quick insights and decision-making.

In [ ]:
WEIGHTS = {
    "compliance_rate": 0.40,
    "delivery_rate": 0.30,
    "avg_price_efficiency": 0.20,
    "quantity_share": 0.10
}

agg["supplier_score"] = (
    WEIGHTS["compliance_rate"] * agg["compliance_rate"] +
    WEIGHTS["delivery_rate"] * agg["delivery_rate"] +
    WEIGHTS["avg_price_efficiency"] * (1 - (agg["avg_price_efficiency"] / agg["avg_price_efficiency"].max())) +
    WEIGHTS["quantity_share"] * agg["quantity_share"]
)

# --- Ranking ---
agg["rank"] = agg["supplier_score"].rank(ascending=False)
agg = agg.sort_values("rank")

# Move rank to 2nd column (right after the first column)
#rank_col = agg.pop("rank")   # remove rank temporarily
# agg.insert(1, "rank", rank_col)  # insert at index 1 (second column)

print("\nTop 10 suppliers by rank:")
print(agg.head(10))


Top 10 suppliers by rank:
          Supplier  compliance_rate  delivery_rate  avg_price_efficiency  \
3    Epsilon_Group         0.981928       0.722892             59.392711   
0        Alpha_Inc         0.936170       0.758865             58.386525   
4         Gamma_Co         0.860140       0.720280             59.348881   
1    Beta_Supplies         0.756410       0.705128             58.279423   
2  Delta_Logistics         0.608187       0.701754             56.236023   

   quantity_share  total_orders  total_quantity  supplier_score  rank  cluster  
3        0.211690           166          180053        0.630808   1.0        1  
0        0.176843           141          150414        0.623200   2.0        0  
4        0.185663           143          157916        0.578854   3.0        0  
1        0.212234           156          180516        0.539075   4.0        1  
2        0.213570           171          181652        0.485788   5.0        1  


💾 Step 8: Save Model Artifacts & Results

We save the final outputs for future analysis and Streamlit integration:
- supplier_ranking.csv → Ranked supplier list with scores and clusters.
- supplier_model.pkl → Final trained KMeans clustering model.
- scaler.pkl → Scaler used for feature normalization. 
- The step also prints the optimal number of clusters (k) and the Top 10 ranked suppliers for quick verification.

In [ ]:
# --- Save Outputs for Streamlit ---
agg.to_csv("supplier_ranking.csv", index=False)
joblib.dump(final_kmeans, "supplier_model.pkl")
joblib.dump(scaler, "scaler.pkl") 

# Print confirmation and key insights
print("\n✅ Files saved successfully:")
print(" - supplier_ranking.csv")
print(" - supplier_model.pkl")
print(" - scaler.pkl")

print(f"\nBest k (clusters): {best_k}")
print(f"Top 10 suppliers:\n{agg.head(10)}")



✅ Files saved successfully:
 - supplier_ranking.csv
 - supplier_model.pkl
 - scaler.pkl

Best k (clusters): 2
Top 10 suppliers:
          Supplier  compliance_rate  delivery_rate  avg_price_efficiency  \
3    Epsilon_Group         0.981928       0.722892             59.392711   
0        Alpha_Inc         0.936170       0.758865             58.386525   
4         Gamma_Co         0.860140       0.720280             59.348881   
1    Beta_Supplies         0.756410       0.705128             58.279423   
2  Delta_Logistics         0.608187       0.701754             56.236023   

   quantity_share  total_orders  total_quantity  supplier_score  rank  cluster  
3        0.211690           166          180053        0.630808   1.0        1  
0        0.176843           141          150414        0.623200   2.0        0  
4        0.185663           143          157916        0.578854   3.0        0  
1        0.212234           156          180516        0.539075   4.0        1  
2        

## 🚀 Next Steps
1. Use the `supplier_model.pkl` to score **new suppliers**.
2. Visualize clusters with **Plotly** or **Matplotlib**.
3. Build a **Streamlit dashboard** to interactively explore results.
